In [1]:
import json
import os

from skimage.io import imread

from keras import backend as K
from keras.optimizers import Adam
from segmentation_models import losses
from keras.models import load_model

from efficientnet.keras import EfficientNetB0
from efficientnet.keras import preprocess_input

from utils import preprocess_data, make_image_generator


Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
binary_focal_loss = losses.BinaryFocalLoss(alpha=10.0)
dice_loss = losses.DiceLoss(smooth=1)


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)


def binary_focal_dice_loss(y_true, y_pred):
    return binary_focal_loss(y_true, y_pred) + dice_loss(y_true, y_pred)


In [5]:
with open('config.json', 'r') as file:
    configs = json.load(file)
    
model_configs = configs['UNET']

batch_size = model_configs['BATCH_SIZE']
steps_per_epoch = model_configs['STEPS_PER_EPOCH']
img_size = model_configs['IMG_SIZE']
learning_rate = model_configs['LEARNING_RATE']
path_checkpoint = model_configs['PATH_CHECKPOINT']
model_to_load = model_configs['LOAD_MODEL']
    
data_dir = configs["DATA_DIRECTORY"]
train_image_dir = os.path.join(data_dir, 'train_v2')

corrupted_images = configs['EXCLUDE_IMAGES']
train_df, test_df = preprocess_data(os.path.join(data_dir, 'train_ship_segmentations_v2.csv.zip'), corrupted_images)

K.set_image_data_format('channels_last')

In [6]:
model = load_model(f'{path_checkpoint}/{model_to_load}.hdf5', compile=False)

model.compile(
    optimizer=Adam(lr=learning_rate),
    loss=binary_focal_dice_loss,
    metrics=[dice_coef],
)


In [7]:
train_eval_generator = make_image_generator(
    train_df,
    batch_size=batch_size,
    img_size=img_size,
    train_image_dir=train_image_dir,
    preprocessing_function=preprocess_input
)

In [8]:
model.evaluate_generator(train_eval_generator, steps=steps_per_epoch)

C:\Python37\lib\site-packages\keras\utils\data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


[0.10607115924358368, 0.7279453873634338]

In [ ]:
test_eval_generator = make_image_generator(
    test_df,
    batch_size=batch_size,
    img_size=img_size,
    train_image_dir=train_image_dir,
    preprocessing_function=preprocess_input
)

In [10]:
model.evaluate_generator(test_eval_generator, steps=steps_per_epoch)

[0.11424889415502548, 0.7290158867835999]